In [1]:
import ps, ps_utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
ps_utils.switch_to_dev()
ps_utils.global_algo_id()

[ERROR] 2023-06-04 15:24:04,906 - You do not have xbbg installed, some functionality will not be available
[INFO] 2023-06-04 15:24:05,334 - ps client switched from prod_pvlss to dev


'f5295f4beeee3193f84ba932a99208d55c13f042'

In [ ]:
# Прайсим на 18 мая 2023, спот 80.28625

# Модельный запрос

In [53]:
r=ps.get('aa95654b49ef26517abfa2899376601a222c20ea')
r.Model.MarketDataSet=ps_utils.get_mds(datetime.date(2023,5,18))
r.Model.MarketDataSet.Correlations['RUB_KEY_RATE&USD_LIBOR_3M']=0.0
r.Product.CounterpartyCreditRating=8
r.Product.NettingSets[0].MarginSets[0].Products=['4f52be38e98dd17b73917066518f0ed8483f461f']
ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')

[INFO] 2023-06-05 14:28:41,731 - Compute:ef6d861d97dee6c6dfd214e08974e88a446f8194->86b3ba56f78b201d35cacf2ace642de41bca6af7


Dict([('BCVA', 0.8817287983619622),
      ('CVA', -57.97024418162306),
      ('DVA', 58.85197297998502),
      ('ProductValue', -251.14736432968456),
      ('ReportingCurrency', 'USD'),
      ('StandardError', 235.53668454888796),
      ('CashToSettleToday', 0.0),
      ('CVA_CurrencyExposure',
       Dict([('RUB', -1319.6267416914914), ('USD', 1261.6564975098684)])),
      ('DVA_CurrencyExposure',
       Dict([('RUB', -1276.3214604488421), ('USD', 1335.1734334288271)])),
      ('CVA_ParallelForeignExchangeVega',
       Dict([('USDRUB', -354.57782122074525)])),
      ('DVA_ParallelForeignExchangeVega',
       Dict([('USDRUB', 356.9931057882114)]))])

# 1) на 3 месяца atmf 81.396

In [68]:
r=ps.get('ef6d861d97dee6c6dfd214e08974e88a446f8194')
mas3m=[]


r.Product.NettingSets[0].MarginSets[0].Products[0].FixingDate=datetime.date(2023,8,21)
r.Product.NettingSets[0].MarginSets[0].Products[0].SettlementDate=datetime.date(2023,8,21)

for i in range(10,26):
    r.Product.CounterpartyCreditRating=i
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike=81.396
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike+=0.3
    z=ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
    
    Delta=np.abs(z['CVA_CurrencyExposure']['USD']+z['DVA_CurrencyExposure']['USD'])
    Vega=np.abs(z['CVA_ParallelForeignExchangeVega']['USDRUB']+z['DVA_ParallelForeignExchangeVega']['USDRUB'])/100
    
    bcva=(np.abs(z['BCVA']))
    bcva+= Delta*0.05 + Vega*3
    bcva_in_kops=bcva/np.abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
#     print("Not all-in")
#     print("Rating=", r.Product.CounterpartyCreditRating)
#     print("Z[bcva] in usd=", np.abs(z['BCVA']), np.abs(z['BCVA'])/np.abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike)
#     print("BCva with delta and Vega in usd=", bcva, bcva_in_kops)
#     print('Delta=',z['CVA_CurrencyExposure']['USD']+z['DVA_CurrencyExposure']['USD'])
#     print("Vega=", (z['CVA_ParallelForeignExchangeVega']['USDRUB']+z['DVA_ParallelForeignExchangeVega']['USDRUB'])/100)
    
    
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike+=bcva_in_kops
    z=ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
    bcva_in_kops=z['BCVA']/r.Product.NettingSets[0].MarginSets[0].Products[0].Notional*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
#     print("All-in")
#     print("Rating=", r.Product.CounterpartyCreditRating)
#     print("BCVa in usd=", z['BCVA'])
#     print("Bcva in kops=",bcva_in_kops)
    mas3m.append(np.abs(z['BCVA']))
print(mas3m)

[INFO] 2023-06-05 16:51:20,947 - Compute:041e3d511e070b066b97028a1df00d4254903cd3->901b3e96482b3e46660c18fd010fea40b222902f
[INFO] 2023-06-05 16:51:21,056 - Compute:d7cb18b398255bed595c0bf8ce4a4e845dd98c3d->548a1ac471197c8ce0124e25650621e949622a97
[INFO] 2023-06-05 16:51:21,095 - Compute:6427eb93f152632d7df85e29c40ea78ed28afa1d->e3066272d8721c47b42d33f87374c1475895a273
[INFO] 2023-06-05 16:51:21,193 - Compute:ef1f480196598bd0039a178e0171ba18e068ed14->0e1911a262d934e4d253ad0c1986628b53c469d7
[INFO] 2023-06-05 16:51:21,239 - Compute:07b70b0673d1a90f05b028274421767e4d3de54b->69731fd7866109ec10894df544e77c884b3a5bef
[INFO] 2023-06-05 16:51:21,339 - Compute:76a7f101f26ce7159bffcf1aaf848b411f9a68e1->70f8882c4cba370bae05ecc38ef396e9dfdea0e7
[INFO] 2023-06-05 16:51:21,380 - Compute:ba408b793188b5bc19fe5cf754d38c8fdb82d93e->ffe4d6d29bbc26b3bdda4eb6ab4dd9d45b5471e1
[INFO] 2023-06-05 16:51:21,478 - Compute:f551a1f0996907d38efd52c41fffc1310248ece5->6f7621fab9832ea3649813aa24a9ac92c9661a26
[INFO] 2

[19.796333765443705, 28.320373969300036, 40.16332156998611, 56.416236962537646, 78.80034092508873, 109.65273388140311, 152.2321413113285, 210.96880214635493, 292.01939268205604, 403.92301063474713, 558.6836086089196, 772.9589565020783, 1069.9404316086836, 1482.4068247312152, 2056.1422615280712, 2855.1720423453125]


# 2) На 6 месяцев atmf =82.1696

In [70]:
r=ps.get('ef6d861d97dee6c6dfd214e08974e88a446f8194')
mas6m=[]

r.Product.NettingSets[0].MarginSets[0].Products[0].FixingDate=datetime.date(2023,11,20)
r.Product.NettingSets[0].MarginSets[0].Products[0].SettlementDate=datetime.date(2023,11,20)

for i in range(10,26):
    r.Product.CounterpartyCreditRating=i
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike=82.1696
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike+=0.3
    z=ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
    
    Delta=np.abs(z['CVA_CurrencyExposure']['USD']+z['DVA_CurrencyExposure']['USD'])
    Vega=np.abs(z['CVA_ParallelForeignExchangeVega']['USDRUB']+z['DVA_ParallelForeignExchangeVega']['USDRUB'])/100
    
    bcva=(np.abs(z['BCVA']))
    bcva+= Delta*0.05 + Vega*3
    bcva_in_kops=bcva/np.abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
#     print("Not all-in")
#     print("Rating=", r.Product.CounterpartyCreditRating)
#     print("Z[bcva] in usd=", np.abs(z['BCVA']), np.abs(z['BCVA'])/np.abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike)
#     print("BCva with delta and Vega in usd=", bcva, bcva_in_kops)
#     print('Delta=',z['CVA_CurrencyExposure']['USD']+z['DVA_CurrencyExposure']['USD'])
#     print("Vega=", (z['CVA_ParallelForeignExchangeVega']['USDRUB']+z['DVA_ParallelForeignExchangeVega']['USDRUB'])/100)
    
    
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike+=bcva_in_kops
    z=ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
    bcva_in_kops=z['BCVA']/r.Product.NettingSets[0].MarginSets[0].Products[0].Notional*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
#     print("All-in")
#     print("Rating=", r.Product.CounterpartyCreditRating)
#     print("BCVa in usd=", z['BCVA'])
#     print("Bcva in kops=",bcva_in_kops)
    
    mas6m.append(np.abs(z['BCVA']))
print(mas6m)

[INFO] 2023-06-05 17:01:11,084 - Compute:2fa2227cd3c680f300252f9de25cfad3bf9a99c9->be179ca14a02610e54cd689c1645c3b8a34ea236
[INFO] 2023-06-05 17:01:11,195 - Compute:f41c85817b7bc6c77c64668ce32a3cff8f91fbad->42ca93926f4fde57a955c52b38ae38156344e140
[INFO] 2023-06-05 17:01:11,255 - Compute:a87c81e10fd0519a326ae9c3b81d1df85b64af71->473734dc17858f1e26f1df21c988fe0f13214cd4
[INFO] 2023-06-05 17:01:11,357 - Compute:b307cc59bd04588607b611b6f35be3dd13efcc3e->ee033a4dbc7826781820fab9574ee94053f20b02
[INFO] 2023-06-05 17:01:11,415 - Compute:8e305f57488e69037aab421d8ffffecdb0612893->76043cb314dbe60a3db91a769503024d6d8342cb
[INFO] 2023-06-05 17:01:11,521 - Compute:09f4cf9f99672fb369b4ffee0a874d20caf4c080->4438279b9dd8926271d527b762aff6c68cb53ae2
[INFO] 2023-06-05 17:01:11,582 - Compute:f600febc2c74a3e8d9c60445ad13c2842920702d->7cba6e736eb5bace7e5f9954cf0d1470986354dd
[INFO] 2023-06-05 17:01:11,683 - Compute:9abb37145a30927db4a5eaadce0bcc020380fc68->a4e851d91c1f7092d99092dc24c00e46fa925282
[INFO] 2

[48.715256020333584, 71.94344378329168, 104.21532447804546, 148.50342289281136, 209.49655865146218, 293.5597151760447, 409.5639919011419, 569.5599573119848, 790.2696846780921, 1094.825841895578, 1515.5945000028803, 2097.100118045925, 2900.3559912546075, 4009.2129298429445, 5534.882135928233, 7618.944183311654]


# 3) 1y atmf=84.0497

In [78]:
r=ps.get('ef6d861d97dee6c6dfd214e08974e88a446f8194')
mas1y=[]

r.Product.NettingSets[0].MarginSets[0].Products[0].FixingDate=datetime.date(2024,5,20)
r.Product.NettingSets[0].MarginSets[0].Products[0].SettlementDate=datetime.date(2024,5,20)

for i in range(10,26):
    r.Product.CounterpartyCreditRating=i
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike=84.0497
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike+=0.3
    z=ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
    
    Delta=np.abs(z['CVA_CurrencyExposure']['USD']+z['DVA_CurrencyExposure']['USD'])
    Vega=np.abs(z['CVA_ParallelForeignExchangeVega']['USDRUB']+z['DVA_ParallelForeignExchangeVega']['USDRUB'])/100
    
    bcva=(np.abs(z['BCVA']))
    bcva+= Delta*0.05 + Vega*3
    bcva_in_kops=bcva/np.abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
#     print("Not all-in")
#     print("Rating=", r.Product.CounterpartyCreditRating)
#     print("Z[bcva] in usd=", np.abs(z['BCVA']), np.abs(z['BCVA'])/np.abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike)
#     print("BCva with delta and Vega in usd=", bcva, bcva_in_kops)
#     print('Delta=',z['CVA_CurrencyExposure']['USD']+z['DVA_CurrencyExposure']['USD'])
#     print("Vega=", (z['CVA_ParallelForeignExchangeVega']['USDRUB']+z['DVA_ParallelForeignExchangeVega']['USDRUB'])/100)
    
    
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike+=bcva_in_kops
    z=ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
    bcva_in_kops=z['BCVA']/r.Product.NettingSets[0].MarginSets[0].Products[0].Notional*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
#     print("All-in")
#     print("Rating=", r.Product.CounterpartyCreditRating)
#     print("BCVa in usd=", z['BCVA'])
#     print("Bcva in kops=",bcva_in_kops)
    
    mas1y.append(np.abs(z['BCVA']))
print(mas1y)

[INFO] 2023-06-05 18:42:16,345 - Compute:28c1da9bfedb68c0f4027a59ddaf552c1c6b13e7->9ee16b31a40d75610278fe450eb73957e9b79bbd
[INFO] 2023-06-05 18:42:24,067 - Compute:f3982d783a9c9fae1a814c532bc2dfa1df8e1f9d->18f24a2a397eb56f133264076e2460dc33dcf9cc
[INFO] 2023-06-05 18:42:24,365 - Compute:8084439ae06f706c5d4204731a0a03c5014fe73d->3456b71aa543f5b97430f98fb614018061787ec3
[INFO] 2023-06-05 18:42:24,455 - Compute:d63bcd2fbde112f7642cbcda580e8a3948d2cfb0->5bfd061193a3f442350ce07378d576b9d5948af1
[INFO] 2023-06-05 18:42:24,503 - Compute:364f284e126d055f43663151914e7de35ba29984->65b40ba57d1d47ed7eb326850ec2f1cb57bf14e2
[INFO] 2023-06-05 18:42:24,599 - Compute:2ea5087b51574cd5e1e22988aa43e7a657be335e->2b26eb59e83f19c4016c2e7cba04f4015f43ece4
[INFO] 2023-06-05 18:42:24,658 - Compute:14f9f0592ae4295c2e4715d22f3ea85a20e56366->ccdf7ad94345e8f20109f0efb5ba1192fdbd3720
[INFO] 2023-06-05 18:42:24,754 - Compute:d2314ffc6adb636af30ec51c029a8a47afcb2d3d->e7f658eb056317600083772e2a941f95e1cfea2b
[INFO] 2

[124.19129349627212, 192.94773861363, 288.4434235241985, 419.4374489497435, 599.7267189361046, 847.9834695412992, 1190.121301265126, 1661.1023146919913, 2308.958772892823, 3199.088282928823, 4420.727557834911, 6091.504727547538, 8361.354380939765, 11411.931066098252, 15431.106313004302, 20549.705562577048]


# 4) 2y atmf 89.0323

In [79]:
r=ps.get('ef6d861d97dee6c6dfd214e08974e88a446f8194')
mas2y=[]

r.Product.NettingSets[0].MarginSets[0].Products[0].FixingDate=datetime.date(2025,5,20)
r.Product.NettingSets[0].MarginSets[0].Products[0].SettlementDate=datetime.date(2025,5,20)

for i in range(10,26):
    r.Product.CounterpartyCreditRating=i
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike=89.0323
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike+=0.3
    z=ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
    
    Delta=np.abs(z['CVA_CurrencyExposure']['USD']+z['DVA_CurrencyExposure']['USD'])
    Vega=np.abs(z['CVA_ParallelForeignExchangeVega']['USDRUB']+z['DVA_ParallelForeignExchangeVega']['USDRUB'])/100
    
    bcva=(np.abs(z['BCVA']))
    bcva+= Delta*0.05 + Vega*3
    bcva_in_kops=bcva/np.abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
#     print("Not all-in")
#     print("Rating=", r.Product.CounterpartyCreditRating)
#     print("Z[bcva] in usd=", np.abs(z['BCVA']), np.abs(z['BCVA'])/np.abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike)
#     print("BCva with delta and Vega in usd=", bcva, bcva_in_kops)
#     print('Delta=',z['CVA_CurrencyExposure']['USD']+z['DVA_CurrencyExposure']['USD'])
#     print("Vega=", (z['CVA_ParallelForeignExchangeVega']['USDRUB']+z['DVA_ParallelForeignExchangeVega']['USDRUB'])/100)
    
    
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike+=bcva_in_kops
    z=ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
    bcva_in_kops=z['BCVA']/r.Product.NettingSets[0].MarginSets[0].Products[0].Notional*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
#     print("All-in")
#     print("Rating=", r.Product.CounterpartyCreditRating)
#     print("BCVa in usd=", z['BCVA'])
#     print("Bcva in kops=",bcva_in_kops)
    
    mas2y.append(np.abs(z['BCVA']))
print(mas2y)

[INFO] 2023-06-05 18:42:26,600 - Compute:d5d5c3e6c802ee80070937bed319d155dbe7f213->5d695848e45a8ca3c07bf4a51a9cd6bc5161a54e
[INFO] 2023-06-05 18:42:26,692 - Compute:eeb5704e3a04eab58bbad3c1f4b857fe53f78bb2->9a7150cd7fc874cb6c65bbcc36f621cf7a083fa2
[INFO] 2023-06-05 18:42:26,746 - Compute:8b480524e6ab0d3618a86bfb5f975855d798400d->86ac6c8a41c0c572482ff20b2c267633b05df6f9
[INFO] 2023-06-05 18:42:26,844 - Compute:3b0051a9eb133ecb0fff51d63dd35b4621c9d1d5->9304592fb366523aca6cc3f6610049a42f8ac5fd
[INFO] 2023-06-05 18:42:26,898 - Compute:0cb991c7b69e54930793e095c9e7a48804119450->dc63159f97588570cda75afdcd1bf9418a5fb45b
[INFO] 2023-06-05 18:42:27,008 - Compute:ef1ee0a7486cf894efff730ad759676a2110adaa->cc611e4a52ff94397e7ef69bee1eae2c87ebcd34
[INFO] 2023-06-05 18:42:28,577 - Compute:eab019c0b33d08b5425fdce55d472c2bbd22edbd->f6fc10af9a924e686453f766777ee76192e3c761
[INFO] 2023-06-05 18:42:28,824 - Compute:a23d970df7bb1bd2aa5ea76964f8d92fdedc05fa->444b744d50cb722bf5121c721988ebc59241d342
[INFO] 2

[338.6790508651857, 545.965888099272, 833.611718201292, 1227.6901066057903, 1769.1032656876314, 2512.721102309569, 3533.7335434645197, 4931.515765958213, 6838.368511458326, 9425.363205838823, 12906.718317694194, 17523.000161223365, 23493.743948437645, 30913.17099385422, 39538.03916891249, 48520.05557194527]


# 5) 3y atmf=95.493

In [80]:
r=ps.get('ef6d861d97dee6c6dfd214e08974e88a446f8194')
mas3y=[]

r.Product.NettingSets[0].MarginSets[0].Products[0].FixingDate=datetime.date(2026,5,20)
r.Product.NettingSets[0].MarginSets[0].Products[0].SettlementDate=datetime.date(2026,5,20)

for i in range(10,26):
    r.Product.CounterpartyCreditRating=i
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike=95.493
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike+=0.3
    z=ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
    
    Delta=np.abs(z['CVA_CurrencyExposure']['USD']+z['DVA_CurrencyExposure']['USD'])
    Vega=np.abs(z['CVA_ParallelForeignExchangeVega']['USDRUB']+z['DVA_ParallelForeignExchangeVega']['USDRUB'])/100
    
    bcva=(np.abs(z['BCVA']))
    bcva+= Delta*0.05 + Vega*3
    bcva_in_kops=bcva/np.abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
#     print("Not all-in")
#     print("Rating=", r.Product.CounterpartyCreditRating)
#     print("Z[bcva] in usd=", np.abs(z['BCVA']), np.abs(z['BCVA'])/np.abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike)
#     print("BCva with delta and Vega in usd=", bcva, bcva_in_kops)
#     print('Delta=',z['CVA_CurrencyExposure']['USD']+z['DVA_CurrencyExposure']['USD'])
#     print("Vega=", (z['CVA_ParallelForeignExchangeVega']['USDRUB']+z['DVA_ParallelForeignExchangeVega']['USDRUB'])/100)
    
    
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike+=bcva_in_kops
    z=ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
    bcva_in_kops=z['BCVA']/r.Product.NettingSets[0].MarginSets[0].Products[0].Notional*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
#     print("All-in")
#     print("Rating=", r.Product.CounterpartyCreditRating)
#     print("BCVa in usd=", z['BCVA'])
#     print("Bcva in kops=",bcva_in_kops)
    mas3y.append(np.abs(z['BCVA']))
print(mas3y)

[INFO] 2023-06-05 18:42:30,802 - Compute:47a071520a1869e6fa1b67cc7ed245340873c8a6->16f427188db8031ada312de61682d50fb6beedc2
[INFO] 2023-06-05 18:42:30,890 - Compute:5ccbda1bd95cd12505d8db36ca50763d21af3a91->ee8612f1307ceba8373f0c4bbf65170948859f4e
[INFO] 2023-06-05 18:42:30,947 - Compute:a599ffe2bd70bedc27710dc3da82530867f90410->348483a15bf21c22b4d17c69a1ba2049695b5510
[INFO] 2023-06-05 18:42:31,043 - Compute:b44cf1c6745d98fff8222f78133477c9d456802e->3940b51a859c3cf25f228d22429a2bc6fc5a536c
[INFO] 2023-06-05 18:42:31,098 - Compute:7f258781d58b3ea87af300f98933fe824b2bb70d->d27248908c73b71e95fc06e80c54134742342cdd
[INFO] 2023-06-05 18:42:31,191 - Compute:241097e09678db0e45ef7edf0fef988a6f3b573b->7cfbda7371edd9a7bf2081d523472ecd248b3228
[INFO] 2023-06-05 18:42:31,244 - Compute:a96b4f3e67e97ac919f10b4257901495d16d13dc->c95e0812fa88cd957944c2be6f713ec540d2c316
[INFO] 2023-06-05 18:42:31,338 - Compute:2d9a1bdc943e9ab07215ec92b9ff6739f68deaa2->e59e239641497d22667010853576579a346a0af6
[INFO] 2

[605.5785202578968, 993.0098212710518, 1530.109196230845, 2264.911143492325, 3272.4008098431823, 4652.0954571192815, 6538.215218507432, 9103.415128906048, 12567.817611039805, 17195.044645627764, 23270.928238541208, 31020.162307145838, 40441.915354586774, 51047.271044201036, 61538.7278773149, 69762.5179276915]


# 4y atmf=103.0088

In [81]:
r=ps.get('ef6d861d97dee6c6dfd214e08974e88a446f8194')
mas4y=[]

r.Product.NettingSets[0].MarginSets[0].Products[0].FixingDate=datetime.date(2027,5,20)
r.Product.NettingSets[0].MarginSets[0].Products[0].SettlementDate=datetime.date(2027,5,20)

for i in range(10,26):
    r.Product.CounterpartyCreditRating=i
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike=103.0088
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike+=0.3
    z=ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
    
    Delta=np.abs(z['CVA_CurrencyExposure']['USD']+z['DVA_CurrencyExposure']['USD'])
    Vega=np.abs(z['CVA_ParallelForeignExchangeVega']['USDRUB']+z['DVA_ParallelForeignExchangeVega']['USDRUB'])/100
    
    bcva=(np.abs(z['BCVA']))
    bcva+= Delta*0.05 + Vega*3
    bcva_in_kops=bcva/np.abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
#     print("Not all-in")
#     print("Rating=", r.Product.CounterpartyCreditRating)
#     print("Z[bcva] in usd=", np.abs(z['BCVA']), np.abs(z['BCVA'])/np.abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike)
#     print("BCva with delta and Vega in usd=", bcva, bcva_in_kops)
#     print('Delta=',z['CVA_CurrencyExposure']['USD']+z['DVA_CurrencyExposure']['USD'])
#     print("Vega=", (z['CVA_ParallelForeignExchangeVega']['USDRUB']+z['DVA_ParallelForeignExchangeVega']['USDRUB'])/100)
    
    
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike+=bcva_in_kops
    z=ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
    bcva_in_kops=z['BCVA']/r.Product.NettingSets[0].MarginSets[0].Products[0].Notional*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
#     print("All-in")
#     print("Rating=", r.Product.CounterpartyCreditRating)
#     print("BCVa in usd=", z['BCVA'])
#     print("Bcva in kops=",bcva_in_kops)
    mas4y.append(np.abs(z['BCVA']))
print(mas4y)

[INFO] 2023-06-05 18:42:33,451 - Compute:46736b9c35bb1c58db9055696d17f8ab71cdf812->7124bf4e6aa11985dbf2537d2fc4da7e2a201998
[INFO] 2023-06-05 18:42:33,540 - Compute:328c641fed375bb7e1747b7092ca88665f2e3fdf->3255bec64187e5e4858eea5ca14785c2166d3f7b
[INFO] 2023-06-05 18:42:33,589 - Compute:64ce587f8d152709bd9e640940ac03a27bcd5c27->32479a7d5be9d34ede3d6f22abd2d06f361055e5
[INFO] 2023-06-05 18:42:33,685 - Compute:aa5c6f57567a94611fd64432befb662b9d8d69d0->e0a1dd99297e5ecc235e4e0960edd429dd99948c
[INFO] 2023-06-05 18:42:33,738 - Compute:503b87cb29522b0e89823a4b0ec05fc467214bb5->0d201b9c0598aab127f4d62e02aac2a756fbcd80
[INFO] 2023-06-05 18:42:33,827 - Compute:6dcdbb5b27d272dd079591f4ddfcddc3b4127ffe->0210ee6ef9e074be261c635be17704483af10ba1
[INFO] 2023-06-05 18:42:33,882 - Compute:bd02ff07ae7d86939263dbdbc58a857fb4cbb068->9662215c81189386ba5d0ed8f9060cdc60831b43
[INFO] 2023-06-05 18:42:33,980 - Compute:797788c9ff95c19e40ec26959a31b2c67448d824->f731aa1fa3f847013a8e0faf833fe7bddcb2122d
[INFO] 2

[884.7348826274683, 1483.5385320842233, 2312.6835384479273, 3445.110252176005, 4993.991966545057, 7107.505644598228, 9981.455118653426, 13858.891619111437, 19031.572068491423, 25810.088669960533, 34449.34471016412, 44962.30705160203, 56824.531330854144, 68645.15378357678, 78056.82982165743, 82377.4589371357]


# 5y atmf=110.9235

In [82]:
r=ps.get('ef6d861d97dee6c6dfd214e08974e88a446f8194')
mas5y=[]

r.Product.NettingSets[0].MarginSets[0].Products[0].FixingDate=datetime.date(2028,5,20)
r.Product.NettingSets[0].MarginSets[0].Products[0].SettlementDate=datetime.date(2028,5,20)

for i in range(10,26):
    r.Product.CounterpartyCreditRating=i
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike=110.9235
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike+=0.3
    z=ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
    
    Delta=np.abs(z['CVA_CurrencyExposure']['USD']+z['DVA_CurrencyExposure']['USD'])
    Vega=np.abs(z['CVA_ParallelForeignExchangeVega']['USDRUB']+z['DVA_ParallelForeignExchangeVega']['USDRUB'])/100
    
    bcva=(np.abs(z['BCVA']))
    bcva+= Delta*0.05 + Vega*3
    bcva_in_kops=bcva/np.abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
#     print("Not all-in")
#     print("Rating=", r.Product.CounterpartyCreditRating)
#     print("Z[bcva] in usd=", np.abs(z['BCVA']), np.abs(z['BCVA'])/np.abs(r.Product.NettingSets[0].MarginSets[0].Products[0].Notional)*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike)
#     print("BCva with delta and Vega in usd=", bcva, bcva_in_kops)
#     print('Delta=',z['CVA_CurrencyExposure']['USD']+z['DVA_CurrencyExposure']['USD'])
#     print("Vega=", (z['CVA_ParallelForeignExchangeVega']['USDRUB']+z['DVA_ParallelForeignExchangeVega']['USDRUB'])/100)
    
    
    r.Product.NettingSets[0].MarginSets[0].Products[0].Strike+=bcva_in_kops
    z=ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
    bcva_in_kops=z['BCVA']/r.Product.NettingSets[0].MarginSets[0].Products[0].Notional*r.Product.NettingSets[0].MarginSets[0].Products[0].Strike
#     print("All-in")
#     print("Rating=", r.Product.CounterpartyCreditRating)
#     print("BCVa in usd=", z['BCVA'])
#     print("Bcva in kops=",bcva_in_kops)
    
    mas5y.append(np.abs(z['BCVA']))
print(mas5y)

[INFO] 2023-06-05 18:42:50,588 - Compute:dce521991ca6882b6ff64f37af2ba7df12e8b910->a733cf83071d189a935afeceb377378162341e88
[INFO] 2023-06-05 18:42:50,681 - Compute:f0cee0b2a17c87f6fdb1769898d5cd22dd1962d8->679270ebf4d4aa34b1e4e4d55f2366ad8aafa326
[INFO] 2023-06-05 18:42:50,735 - Compute:ccbbc41f8d48d987c7b257a37b67679e1ef2b06b->25953abe39e9934f08abeb423d6cb00decd82544
[INFO] 2023-06-05 18:42:50,835 - Compute:8c777390139b8dc2bce115c703199577e41aee28->a30e688307b004d1288bf4352d39128f3b99bde6
[INFO] 2023-06-05 18:42:50,889 - Compute:7062f25f06fac725c19773686d902f6b6ec9fe67->4179edecd080890c56b80bfd298dd86d21af667e
[INFO] 2023-06-05 18:42:50,989 - Compute:409875d5a38f72fdd30ab258a5bf0aa25026fdff->04d1594c449137b9b8f654c02aceaa4497a89f9e
[INFO] 2023-06-05 18:42:51,045 - Compute:33809aa710d824b592b2b11cc78fd6bd669c0fd6->9d767f06973575698df128a5432e8b7a87d673d7
[INFO] 2023-06-05 18:42:51,155 - Compute:47f4e8ba48a1daaa4faa6054772ac414ae33ebb4->1feb1f2803af46a35c191e93a7363690aef32f9f
[INFO] 2

[1188.9327387225312, 2020.6348541528696, 3170.659450275386, 4738.197095906172, 6876.022919406027, 9780.876045964693, 13706.17312663028, 18952.2031396069, 25850.370247372593, 34691.228479820464, 45575.5334868245, 58117.08399710263, 71074.41864929177, 82150.63486875269, 88409.35579280126, 87708.41518898754]


In [38]:
df=pd.DataFrame(index=np.arange(8,26))
df['3m']=mas3m
df['6m']=mas6m
df['1y']=mas1y
df['2y']=mas2y
df['3y']=mas3y
df['4y']=mas4y
df['5y']=mas5y
df

,3m,6m,1y,2y,3y,4y,5y
8,8.73,16.42,33.18,65.28,106.04,141.36,148.06
9,13.20,28.53,69.12,173.80,308.51,455.22,584.53
10,19.40,45.34,119.00,324.31,589.14,889.93,"1,188.61"
11,27.86,68.25,186.96,529.22,970.89,"1,480.69","2,008.68"
12,39.60,100.07,281.26,813.21,"1,499.27","2,297.29","3,140.51"
13,55.70,143.68,410.44,"1,201.56","2,220.60","3,409.94","4,679.40"
14,77.85,203.66,587.88,"1,733.80","3,206.74","4,926.92","6,771.10"
15,108.34,286.15,831.57,"2,462.38","4,551.92","6,988.05","9,600.52"
16,150.34,399.68,"1,166.20","3,458.20","6,381.18","9,774.78","13,401.19"
17,208.12,555.66,"1,624.60","4,813.21","8,851.65","13,506.32","18,441.32"


In [83]:
df=pd.DataFrame(index=np.arange(10,26))
df['3m']=mas3m
df['6m']=mas6m
df['1y']=mas1y
df['2y']=mas2y
df['3y']=mas3y
df['4y']=mas4y
df['5y']=mas5y
df

,3m,6m,1y,2y,3y,4y,5y
10,19.80,48.72,124.19,338.68,605.58,884.73,"1,188.93"
11,28.32,71.94,192.95,545.97,993.01,"1,483.54","2,020.63"
12,40.16,104.22,288.44,833.61,"1,530.11","2,312.68","3,170.66"
13,56.42,148.50,419.44,"1,227.69","2,264.91","3,445.11","4,738.20"
14,78.80,209.50,599.73,"1,769.10","3,272.40","4,993.99","6,876.02"
15,109.65,293.56,847.98,"2,512.72","4,652.10","7,107.51","9,780.88"
16,152.23,409.56,"1,190.12","3,533.73","6,538.22","9,981.46","13,706.17"
17,210.97,569.56,"1,661.10","4,931.52","9,103.42","13,858.89","18,952.20"
18,292.02,790.27,"2,308.96","6,838.37","12,567.82","19,031.57","25,850.37"
19,403.92,"1,094.83","3,199.09","9,425.36","17,195.04","25,810.09","34,691.23"


In [33]:
r.Queries=['CVA_CurrencyExposure']
ps_utils.compute(ps.put(r),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')

[INFO] 2023-06-04 15:50:17,510 - Compute:ad3c67f73a9fe382eced08262545c0116c4b6398->accea1955b035532e740a07e7d760b6d5353d484


Dict([('CVA_CurrencyExposure',
       Dict([('RUB', -402884.1336664455), ('USD', 339071.57893402857)]))])

In [45]:
mds2022=ps_utils.get_mds(datetime.date(2022,1,19))
Deltas=mds2022.VolatilitySurfaces['USDRUB'].Deltas
Tenors=mds2022.VolatilitySurfaces['USDRUB'].Tenors
vol2022=pd.DataFrame(columns=Deltas)
for i in range(len(Tenors)):
    vol2022= vol2022.append({ k:v for (k,v) in zip(Deltas, mds2022.VolatilitySurfaces['USDRUB'].VolatilityQuotes[i])}, ignore_index=True)
vol2022

,10RR,25RR,ATM,25BF,10BF
0,0.12,0.06,0.21,0.01,0.03
1,0.11,0.06,0.18,0.01,0.03
2,0.12,0.06,0.18,0.01,0.04
3,0.15,0.07,0.18,0.01,0.04
4,0.13,0.07,0.17,0.01,0.04
5,0.13,0.07,0.17,0.01,0.04
6,0.13,0.06,0.17,0.01,0.04
7,0.13,0.07,0.16,0.01,0.04
8,0.14,0.07,0.16,0.01,0.04
9,0.14,0.07,0.17,0.01,0.04


In [46]:
mds2023=ps_utils.get_mds(datetime.date(2022,5,18))
Deltas=mds2023.VolatilitySurfaces['USDRUB'].Deltas
Tenors=mds2023.VolatilitySurfaces['USDRUB'].Tenors
vol2023=pd.DataFrame(columns=Deltas)
for i in range(len(Tenors)):
    vol2023 = vol2023.append({ k:v for (k,v) in zip(Deltas, mds2023.VolatilitySurfaces['USDRUB'].VolatilityQuotes[i])}, ignore_index=True)
vol2023

,10RR,25RR,ATM,25BF,10BF
0,0.19,0.10,0.25,0.02,0.08
1,0.21,0.11,0.25,0.02,0.08
2,0.24,0.12,0.30,0.03,0.10
3,0.26,0.13,0.33,0.03,0.12
4,0.27,0.14,0.37,0.03,0.12
5,0.28,0.14,0.37,0.03,0.13
6,0.30,0.15,0.37,0.04,0.13
7,0.28,0.14,0.34,0.04,0.13
8,0.26,0.13,0.30,0.04,0.14
9,0.23,0.11,0.30,0.04,0.13


In [50]:
vol2023/vol2022

,10RR,25RR,ATM,25BF,10BF
0,1.68,1.68,1.19,2.79,2.52
1,1.88,1.88,1.43,2.80,2.50
2,1.96,1.96,1.65,3.06,2.61
3,1.81,1.81,1.89,3.20,2.69
4,2.08,2.08,2.14,3.37,2.84
5,2.15,2.15,2.21,3.43,2.89
6,2.32,2.32,2.18,3.71,3.08
7,2.11,2.11,2.07,3.76,3.23
8,1.95,1.95,1.83,3.63,3.49
9,1.65,1.65,1.81,3.33,2.88


In [84]:
df.to_excel('CVA_forForwards19мая2023VD.xlsx')